# 3. Modeling features do Prepare Data

## 3.1 Import the Libs

In [2]:
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup
import time
import pyodbc
import numpy as np
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import matplotlib.pyplot as plt

# 3.2 Reading data From Database

In [3]:
# Function Select
def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'MLS_Project')
    out = pd.read_sql_query(query,conn)
    return out

query_all_rankTeam = '''
    Select 
    * 
    from TableRankingStandardTeamNames
'''
query_all_MatchDetails = '''
    Select 
    * 
    from TableMatchDetails
'''

def SQLTruncate(TableName):
    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    cursor.execute(f'''
    
                   TRUNCATE TABLE {TableName}
                     
                   ''')
    conn.commit()
    cursor.close()

In [4]:
df_rankTeam = SQLSelect(query_all_rankTeam)
df_MatchDetails = SQLSelect(query_all_MatchDetails)

In [80]:
df_rankTeam.sort_values(by='SEASON_RT')

,RK,TEAM_NAME_RT,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,SEASON_RT,W_MP,D_MP,L_MP,GD_MP
54,7,Orlando-City,34,12,8,14,46,56,-10,44,1.29,2015,0.352941,0.235294,0.411765,-0.294118
65,8,Houston-Dynamo,34,11,9,14,42,49,-7,42,1.24,2015,0.323529,0.264706,0.411765,-0.205882
64,7,San-Jose-Earthquakes,34,13,8,13,41,39,2,47,1.38,2015,0.382353,0.235294,0.382353,0.058824
63,6,Sporting-KC,34,14,9,11,48,45,3,51,1.50,2015,0.411765,0.264706,0.323529,0.088235
62,5,LA-Galaxy,34,14,9,11,56,46,10,51,1.50,2015,0.411765,0.264706,0.323529,0.294118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,2,Atlanta-United,6,4,1,1,13,9,4,13,2.17,2023,0.666667,0.166667,0.166667,0.666667
190,1,FC-Cincinnati,6,4,2,0,8,4,4,14,2.33,2023,0.666667,0.333333,0.000000,0.666667
217,13,Sporting-KC,6,0,3,3,2,7,-5,3,0.50,2023,0.000000,0.500000,0.500000,-0.833333
203,14,Charlotte-FC,6,1,2,3,6,11,-5,5,0.83,2023,0.166667,0.333333,0.500000,-0.833333


In [6]:
#Removing unecessary columns
columns_to_remove = ['CONFERENCE','XGA','XG','XGD','XGD_90','NOTES','CONFERENCE','Generalkey']
df_rankTeam = df_rankTeam.drop(columns=columns_to_remove)

In [7]:
df_rankTeam.rename(columns = {'TEAM_NAME':'TEAM_NAME_RT','Rk':'RK','SEASON':'SEASON_RT'}, inplace = True)
df_rankTeam

,RK,TEAM_NAME_RT,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,SEASON_RT
0,1,Philadelphia-Union,23,14,5,4,44,20,24,47,2.04,2020
1,2,Toronto-FC,23,13,5,5,33,26,7,44,1.91,2020
2,3,Columbus-Crew,23,12,5,6,36,21,15,41,1.78,2020
3,4,Orlando-City,23,11,8,4,40,25,15,41,1.78,2020
4,5,New-York-City-FC,23,12,3,8,37,25,12,39,1.70,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
214,10,Portland-Timbers,6,1,2,3,6,11,-5,5,0.83,2023
215,11,Real-Salt-Lake,5,1,0,4,3,13,-10,3,0.60,2023
216,12,LA-Galaxy,5,0,3,2,3,6,-3,3,0.60,2023
217,13,Sporting-KC,6,0,3,3,2,7,-5,3,0.50,2023


# 3.3 Creating Features

In [8]:
#Add new column to df_MatchDetails
df_MatchDetails['SEASON_BEFORE'] = df_MatchDetails['SEASON']-1

In [82]:
#Removing Season 2015, because is the first on Ranking DF and do not have Season before
df_MatchDetails = df_MatchDetails[df_MatchDetails['SEASON']>2015]

In [83]:
df_MatchDetails.sort_values(by='SEASON')

,Generalkey,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE
1453,1454,2016-08-13,20:00:00,Sat,Sporting-KC,FC-Dallas,D,2016,2015
1618,1619,2016-06-18,16:00:00,Sat,New-England-Revolution,Vancouver-Whitecaps-FC,A,2016,2015
1619,1620,2016-06-25,19:00:00,Sat,Vancouver-Whitecaps-FC,Philadelphia-Union,A,2016,2015
1620,1621,2016-07-16,19:00:00,Sat,Orlando-City,Vancouver-Whitecaps-FC,D,2016,2015
1621,1622,2016-07-23,20:30:00,Sat,Vancouver-Whitecaps-FC,Houston-Dynamo,D,2016,2015
...,...,...,...,...,...,...,...,...,...
3201,3202,2023-05-17,19:30:00,Wed,Vancouver-Whitecaps-FC,FC-Dallas,F,2023,2022
3200,3201,2023-05-13,19:30:00,Sat,FC-Dallas,Austin-FC,F,2023,2022
3199,3200,2023-04-22,19:30:00,Sat,FC-Dallas,New-York-City-FC,F,2023,2022
3197,3198,2023-04-08,19:30:00,Sat,FC-Dallas,Inter-Miami,F,2023,2022


In [10]:
#Add new column to df_rankTeam
df_rankTeam['W_MP']  = df_rankTeam['W']/df_rankTeam['MP']
df_rankTeam['D_MP']  = df_rankTeam['D']/df_rankTeam['MP']
df_rankTeam['L_MP']  = df_rankTeam['L']/df_rankTeam['MP']
df_rankTeam['GD_MP'] = df_rankTeam['GD']/df_rankTeam['MP']
df_rankTeam

,RK,TEAM_NAME_RT,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,SEASON_RT,W_MP,D_MP,L_MP,GD_MP
0,1,Philadelphia-Union,23,14,5,4,44,20,24,47,2.04,2020,0.608696,0.217391,0.173913,1.043478
1,2,Toronto-FC,23,13,5,5,33,26,7,44,1.91,2020,0.565217,0.217391,0.217391,0.304348
2,3,Columbus-Crew,23,12,5,6,36,21,15,41,1.78,2020,0.521739,0.217391,0.260870,0.652174
3,4,Orlando-City,23,11,8,4,40,25,15,41,1.78,2020,0.478261,0.347826,0.173913,0.652174
4,5,New-York-City-FC,23,12,3,8,37,25,12,39,1.70,2020,0.521739,0.130435,0.347826,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,10,Portland-Timbers,6,1,2,3,6,11,-5,5,0.83,2023,0.166667,0.333333,0.500000,-0.833333
215,11,Real-Salt-Lake,5,1,0,4,3,13,-10,3,0.60,2023,0.200000,0.000000,0.800000,-2.000000
216,12,LA-Galaxy,5,0,3,2,3,6,-3,3,0.60,2023,0.000000,0.600000,0.400000,-0.600000
217,13,Sporting-KC,6,0,3,3,2,7,-5,3,0.50,2023,0.000000,0.500000,0.500000,-0.833333


In [ ]:
#Merge new columns to df_rankTeam

In [84]:
# Adding HOME team details from last season
df_MatchDetailsV02 = pd.merge(df_MatchDetails,df_rankTeam, how='left',
                              right_on=["TEAM_NAME_RT", "SEASON_RT"], 
                              left_on=["TEAM_HOME", "SEASON_BEFORE"], 
                              suffixes=('_MATCH','_HOME'))

In [85]:
# Rename Specific Columns to HOME
df_MatchDetailsV02.rename(columns = {'RK':'HOME_SB_RK',
                                     'PTS_MP':'HOME_SB_PTS_MP', 
                                     'W_MP':'HOME_SB_W_MP',
                                     'D_MP':'HOME_SB_D_MP',
                                     'L_MP':'HOME_SB_L_MP',
                                     'GD_MP':'HOME_SB_GD_MP',
                                    },
                          inplace = True)
df_MatchDetailsV02.columns

Index(['Generalkey', 'GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY',
       'TEAM_HOME', 'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'TEAM_NAME_RT', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS',
       'HOME_SB_PTS_MP', 'SEASON_RT', 'HOME_SB_W_MP', 'HOME_SB_D_MP',
       'HOME_SB_L_MP', 'HOME_SB_GD_MP'],
      dtype='object')

In [86]:
#Removing unecessary columns
columns_to_remove = ['Generalkey','TEAM_NAME_RT', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS']
df_MatchDetailsV02 = df_MatchDetailsV02.drop(columns=columns_to_remove)
df_MatchDetailsV02

,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,HOME_SB_PTS_MP,SEASON_RT,HOME_SB_W_MP,HOME_SB_D_MP,HOME_SB_L_MP,HOME_SB_GD_MP
0,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,1.00,2019.0,0.235294,0.294118,0.470588,-0.647059
1,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
2,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,1.56,2019.0,0.441176,0.235294,0.323529,0.264706
3,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
4,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,1.24,2019.0,0.352941,0.176471,0.470588,-0.147059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2023-04-29,19:30:00,Sat,Charlotte-FC,DC-United,F,2023,2022,14.0,0.79,2022.0,0.205882,0.176471,0.617647,-1.029412
3193,2023-05-31,19:30:00,Wed,CF-Montreal,DC-United,F,2023,2022,14.0,0.79,2022.0,0.205882,0.176471,0.617647,-1.029412
3194,2023-09-16,19:30:00,Sat,DC-United,Charlotte-FC,F,2023,2022,9.0,1.24,2022.0,0.382353,0.088235,0.529412,-0.235294
3195,2023-06-24,19:30:00,Sat,CF-Montreal,Charlotte-FC,F,2023,2022,9.0,1.24,2022.0,0.382353,0.088235,0.529412,-0.235294


In [87]:
# Adding AWAY team details from last season
df_MatchDetailsV03 = pd.merge(df_MatchDetailsV02,df_rankTeam, how='left',
                              right_on=["TEAM_NAME_RT", "SEASON_RT"], 
                              left_on=["TEAM_AWAY", "SEASON_BEFORE"], 
                              suffixes=('_MATCH','_AWAY'))
df_MatchDetailsV03

,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,HOME_SB_PTS_MP,...,GF,GA,GD,PTS,PTS_MP,SEASON_RT_AWAY,W_MP,D_MP,L_MP,GD_MP
0,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,1.00,...,49.0,67.0,-18.0,38.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
1,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,1.12,...,49.0,59.0,-10.0,40.0,1.18,2019.0,0.352941,0.117647,0.529412,-0.294118
2,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,1.56,...,49.0,67.0,-18.0,38.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
3,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,1.12,...,49.0,59.0,-10.0,40.0,1.18,2019.0,0.352941,0.117647,0.529412,-0.294118
4,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,1.24,...,49.0,67.0,-18.0,38.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2023-04-29,19:30:00,Sat,Charlotte-FC,DC-United,F,2023,2022,14.0,0.79,...,44.0,52.0,-8.0,42.0,1.24,2022.0,0.382353,0.088235,0.529412,-0.235294
3193,2023-05-31,19:30:00,Wed,CF-Montreal,DC-United,F,2023,2022,14.0,0.79,...,63.0,50.0,13.0,65.0,1.91,2022.0,0.588235,0.147059,0.264706,0.382353
3194,2023-09-16,19:30:00,Sat,DC-United,Charlotte-FC,F,2023,2022,9.0,1.24,...,36.0,71.0,-35.0,27.0,0.79,2022.0,0.205882,0.176471,0.617647,-1.029412
3195,2023-06-24,19:30:00,Sat,CF-Montreal,Charlotte-FC,F,2023,2022,9.0,1.24,...,63.0,50.0,13.0,65.0,1.91,2022.0,0.588235,0.147059,0.264706,0.382353


In [88]:
# Rename Specific Columns to HOME
df_MatchDetailsV03.rename(columns = {'RK':'AWAY_SB_RK',
                                     'PTS_MP':'AWAY_SB_PTS_MP', 
                                     'W_MP':'AWAY_SB_W_MP',
                                     'D_MP':'AWAY_SB_D_MP',
                                     'L_MP':'AWAY_SB_L_MP',
                                     'GD_MP':'AWAY_SB_GD_MP',
                                    },
                          inplace = True)
df_MatchDetailsV03.columns

Index(['GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY', 'TEAM_HOME',
       'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'HOME_SB_PTS_MP', 'SEASON_RT_MATCH', 'HOME_SB_W_MP', 'HOME_SB_D_MP',
       'HOME_SB_L_MP', 'HOME_SB_GD_MP', 'AWAY_SB_RK', 'TEAM_NAME_RT', 'MP',
       'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS', 'AWAY_SB_PTS_MP',
       'SEASON_RT_AWAY', 'AWAY_SB_W_MP', 'AWAY_SB_D_MP', 'AWAY_SB_L_MP',
       'AWAY_SB_GD_MP'],
      dtype='object')

In [89]:
#Removing unecessary columns
columns_to_remove = ['SEASON_RT_AWAY','SEASON_RT_MATCH','TEAM_NAME_RT', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS']
df_MatchDetailsV03 = df_MatchDetailsV03.drop(columns=columns_to_remove)
df_MatchDetailsV03

,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,HOME_SB_PTS_MP,HOME_SB_W_MP,HOME_SB_D_MP,HOME_SB_L_MP,HOME_SB_GD_MP,AWAY_SB_RK,AWAY_SB_PTS_MP,AWAY_SB_W_MP,AWAY_SB_D_MP,AWAY_SB_L_MP,AWAY_SB_GD_MP
0,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,1.00,0.235294,0.294118,0.470588,-0.647059,11.0,1.12,0.294118,0.235294,0.470588,-0.529412
1,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,1.12,0.294118,0.235294,0.470588,-0.529412,10.0,1.18,0.352941,0.117647,0.529412,-0.294118
2,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,1.56,0.441176,0.235294,0.323529,0.264706,11.0,1.12,0.294118,0.235294,0.470588,-0.529412
3,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,1.12,0.294118,0.235294,0.470588,-0.529412,10.0,1.18,0.352941,0.117647,0.529412,-0.294118
4,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,1.24,0.352941,0.176471,0.470588,-0.147059,11.0,1.12,0.294118,0.235294,0.470588,-0.529412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2023-04-29,19:30:00,Sat,Charlotte-FC,DC-United,F,2023,2022,14.0,0.79,0.205882,0.176471,0.617647,-1.029412,9.0,1.24,0.382353,0.088235,0.529412,-0.235294
3193,2023-05-31,19:30:00,Wed,CF-Montreal,DC-United,F,2023,2022,14.0,0.79,0.205882,0.176471,0.617647,-1.029412,2.0,1.91,0.588235,0.147059,0.264706,0.382353
3194,2023-09-16,19:30:00,Sat,DC-United,Charlotte-FC,F,2023,2022,9.0,1.24,0.382353,0.088235,0.529412,-0.235294,14.0,0.79,0.205882,0.176471,0.617647,-1.029412
3195,2023-06-24,19:30:00,Sat,CF-Montreal,Charlotte-FC,F,2023,2022,9.0,1.24,0.382353,0.088235,0.529412,-0.235294,2.0,1.91,0.588235,0.147059,0.264706,0.382353


In [90]:
df_MatchDetailsV03.columns

Index(['GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY', 'TEAM_HOME',
       'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'HOME_SB_PTS_MP', 'HOME_SB_W_MP', 'HOME_SB_D_MP', 'HOME_SB_L_MP',
       'HOME_SB_GD_MP', 'AWAY_SB_RK', 'AWAY_SB_PTS_MP', 'AWAY_SB_W_MP',
       'AWAY_SB_D_MP', 'AWAY_SB_L_MP', 'AWAY_SB_GD_MP'],
      dtype='object')

## 3.4 Analsys of NaN infos from Season Before (new teams)

In [91]:
#Count columns with NaN
null_counts = df_MatchDetailsV03.isnull().sum()
null_counts

GAME_Date           0
GAME_Time           0
WEEK_DAY            0
TEAM_AWAY           0
TEAM_HOME           0
RESULT_GAME         0
SEASON              0
SEASON_BEFORE       0
HOME_SB_RK        139
HOME_SB_PTS_MP    139
HOME_SB_W_MP      139
HOME_SB_D_MP      139
HOME_SB_L_MP      139
HOME_SB_GD_MP     139
AWAY_SB_RK        142
AWAY_SB_PTS_MP    142
AWAY_SB_W_MP      142
AWAY_SB_D_MP      142
AWAY_SB_L_MP      142
AWAY_SB_GD_MP     142
dtype: int64

In [93]:
#Gererate columns with NaN
na_list = df_MatchDetailsV03['HOME_SB_RK'].notnull()
df_MatchDetailsV03[-na_list]

,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,HOME_SB_PTS_MP,HOME_SB_W_MP,HOME_SB_D_MP,HOME_SB_L_MP,HOME_SB_GD_MP,AWAY_SB_RK,AWAY_SB_PTS_MP,AWAY_SB_W_MP,AWAY_SB_D_MP,AWAY_SB_L_MP,AWAY_SB_GD_MP
62,2020-10-06,19:30:00,Tue,Minnesota-United,Nashville-SC,D,2020,2019,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.56,0.441176,0.235294,0.323529,0.264706
79,2020-10-20,19:30:00,Tue,FC-Dallas,Nashville-SC,H,2020,2019,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.41,0.382353,0.264706,0.352941,0.235294
83,2020-11-04,19:30:00,Wed,FC-Dallas,Nashville-SC,A,2020,2019,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.41,0.382353,0.264706,0.352941,0.235294
131,2020-09-26,14:30:00,Sat,Houston-Dynamo,Nashville-SC,D,2020,2019,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.18,0.352941,0.117647,0.529412,-0.294118
132,2020-10-10,17:00:00,Sat,Houston-Dynamo,Inter-Miami,H,2020,2019,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.18,0.352941,0.117647,0.529412,-0.294118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727,2023-08-20,20:30:00,Sun,Austin-FC,St-Louis-City,F,2023,2022,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.65,0.470588,0.235294,0.294118,0.470588
2729,2023-08-30,19:30:00,Wed,FC-Dallas,St-Louis-City,F,2023,2022,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.56,0.411765,0.323529,0.264706,0.323529
2733,2023-09-20,19:30:00,Wed,Los-Angeles-FC,St-Louis-City,F,2023,2022,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.97,0.617647,0.117647,0.264706,0.823529
2735,2023-09-30,19:30:00,Sat,Sporting-KC,St-Louis-City,F,2023,2022,NaN,NaN,NaN,NaN,NaN,NaN,12.0,1.18,0.323529,0.205882,0.470588,-0.352941


In [94]:
# Verify the datils of NaN
df_rankTeam[df_rankTeam['TEAM_NAME_RT'] == 'Nashville-SC'].sort_values(by='SEASON_RT')

,RK,TEAM_NAME_RT,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,SEASON_RT,W_MP,D_MP,L_MP,GD_MP
6,7,Nashville-SC,23,8,8,7,24,22,2,32,1.39,2020,0.347826,0.347826,0.304348,0.086957
70,3,Nashville-SC,34,12,18,4,55,33,22,54,1.59,2021,0.352941,0.529412,0.117647,0.647059
133,5,Nashville-SC,34,13,11,10,52,41,11,50,1.47,2022,0.382353,0.323529,0.294118,0.323529
194,5,Nashville-SC,6,3,1,2,6,2,4,10,1.67,2023,0.500000,0.166667,0.333333,0.666667


In [111]:
#Creating a summarized DF to insert on NaN
df_rankTeam_summary = df_rankTeam.groupby('SEASON_RT').agg({'RK': 'max', 
                                                            'PTS_MP': 'mean',
                                                            'W_MP': 'mean',
                                                            'D_MP': 'mean',
                                                            'L_MP': 'mean',
                                                            'GD_MP': 'mean',
                                                           })

df_rankTeam_summary2 = pd.DataFrame()
df_rankTeam_summary2 = pd.DataFrame({
                                    'Max_RK': df_rankTeam_summary['RK']+1,
                                    'Mean_PTS_MP': df_rankTeam_summary['PTS_MP'],
                                    'Mean_W_MP': df_rankTeam_summary['W_MP'],
                                    'Mean_D_MP': df_rankTeam_summary['D_MP'],
                                    'Mean_L_MP': df_rankTeam_summary['L_MP'],
                                    'Mean_GD_MP': df_rankTeam_summary['GD_MP']
                                    })

df_rankTeam_summary2 = df_rankTeam_summary2.reset_index()
df_rankTeam_summary2

,SEASON_RT,Max_RK,Mean_PTS_MP,Mean_W_MP,Mean_D_MP,Mean_L_MP,Mean_GD_MP
0,2015,11,1.394000,0.394118,0.211765,0.394118,2.775558e-18
1,2016,11,1.343000,0.342647,0.314706,0.342647,-2.775558e-18
2,2017,12,1.380000,0.379679,0.240642,0.379679,-1.009294e-17
3,2018,13,1.394348,0.393862,0.212276,0.393862,0.000000e+00
4,2019,13,1.388333,0.387255,0.225490,0.387255,4.625929e-18
5,2020,15,1.385000,0.383011,0.236424,0.380565,5.825093e-03
6,2021,15,1.364444,0.364924,0.270153,0.364924,1.233581e-17
7,2022,15,1.376071,0.376050,0.247899,0.376050,3.965082e-18
8,2023,16,1.350690,0.354023,0.288506,0.357471,-1.839080e-02


In [112]:
#Create a new DF to insert NaN and regard the original Data
df_MatchDetailsV04 = df_MatchDetailsV03.copy()

In [118]:
#Count columns with NaN
null_counts = df_MatchDetailsV04.isnull().sum()
null_counts

GAME_Date           0
GAME_Time           0
WEEK_DAY            0
TEAM_AWAY           0
TEAM_HOME           0
RESULT_GAME         0
SEASON              0
SEASON_BEFORE       0
HOME_SB_RK          0
HOME_SB_PTS_MP      0
HOME_SB_W_MP        0
HOME_SB_D_MP        0
HOME_SB_L_MP        0
HOME_SB_GD_MP       0
AWAY_SB_RK        142
AWAY_SB_PTS_MP    142
AWAY_SB_W_MP      142
AWAY_SB_D_MP      142
AWAY_SB_L_MP      142
AWAY_SB_GD_MP     142
dtype: int64

In [114]:
print(df_MatchDetailsV04.columns)
print('-----------')
print(df_rankTeam_summary2.columns)

Index(['GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY', 'TEAM_HOME',
       'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'HOME_SB_PTS_MP', 'HOME_SB_W_MP', 'HOME_SB_D_MP', 'HOME_SB_L_MP',
       'HOME_SB_GD_MP', 'AWAY_SB_RK', 'AWAY_SB_PTS_MP', 'AWAY_SB_W_MP',
       'AWAY_SB_D_MP', 'AWAY_SB_L_MP', 'AWAY_SB_GD_MP'],
      dtype='object')
-----------
Index(['SEASON_RT', 'Max_RK', 'Mean_PTS_MP', 'Mean_W_MP', 'Mean_D_MP',
       'Mean_L_MP', 'Mean_GD_MP'],
      dtype='object')


In [117]:
# Substitute NaN Home from summary

for index, row in df_MatchDetailsV04.iterrows():
    
    if pd.isnull(row['HOME_SB_RK']):
        #get a season before
        Season_bf = row['SEASON_BEFORE']
        
        # Substitute the value in columns with value from df_rankTeam_summary2
        df_MatchDetailsV04.at[index, 'HOME_SB_RK'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Max_RK']
        df_MatchDetailsV04.at[index, 'HOME_SB_PTS_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_PTS_MP']
        df_MatchDetailsV04.at[index, 'HOME_SB_W_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_W_MP']
        df_MatchDetailsV04.at[index, 'HOME_SB_D_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_D_MP']
        df_MatchDetailsV04.at[index, 'HOME_SB_L_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_L_MP']
        df_MatchDetailsV04.at[index, 'HOME_SB_GD_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_GD_MP']
        



In [119]:
# Substitute NaN Home from summary

for index, row in df_MatchDetailsV04.iterrows():
    
    if pd.isnull(row['AWAY_SB_RK']):
        #get a season before
        Season_bf = row['SEASON_BEFORE']
        
        # Substitute the value in columns with value from df_rankTeam_summary2
        df_MatchDetailsV04.at[index, 'AWAY_SB_RK'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Max_RK']
        df_MatchDetailsV04.at[index, 'AWAY_SB_PTS_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_PTS_MP']
        df_MatchDetailsV04.at[index, 'AWAY_SB_W_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_W_MP']
        df_MatchDetailsV04.at[index, 'AWAY_SB_D_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_D_MP']
        df_MatchDetailsV04.at[index, 'AWAY_SB_L_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_L_MP']
        df_MatchDetailsV04.at[index, 'AWAY_SB_GD_MP'] = df_rankTeam_summary2[df_rankTeam_summary2['SEASON_RT']==Season_bf]['Mean_GD_MP']
        



In [120]:
# Check NaN again
null_counts = df_MatchDetailsV04.isnull().sum()
null_counts

GAME_Date         0
GAME_Time         0
WEEK_DAY          0
TEAM_AWAY         0
TEAM_HOME         0
RESULT_GAME       0
SEASON            0
SEASON_BEFORE     0
HOME_SB_RK        0
HOME_SB_PTS_MP    0
HOME_SB_W_MP      0
HOME_SB_D_MP      0
HOME_SB_L_MP      0
HOME_SB_GD_MP     0
AWAY_SB_RK        0
AWAY_SB_PTS_MP    0
AWAY_SB_W_MP      0
AWAY_SB_D_MP      0
AWAY_SB_L_MP      0
AWAY_SB_GD_MP     0
dtype: int64

## 3.5 Inserting table to the Database

In [121]:
# Create a function to INSERT Table
def SQLInsertTable(TableGameDetails):
    base = pd.DataFrame(columns=['GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY',
                                 'TEAM_HOME', 'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
                                 'HOME_SB_PTS_MP', 'HOME_SB_W_MP', 'HOME_SB_D_MP',
                                 'HOME_SB_L_MP', 'HOME_SB_GD_MP', 'AWAY_SB_RK', 'AWAY_SB_PTS_MP',
                                 'AWAY_SB_W_MP', 'AWAY_SB_D_MP', 'AWAY_SB_L_MP',
                                 'AWAY_SB_GD_MP'])
    
    TableBase = base.append(TableGameDetails) #.fillna(0)

    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    for index, row in TableBase.iterrows():

        cursor.execute('''

            INSERT INTO TableMatchDetailsFull ( 
                GAME_Date,
                GAME_Time,
                WEEK_DAY,
                
                TEAM_AWAY,
                TEAM_HOME,
                RESULT_GAME,
                
                SEASON,
                SEASON_BEFORE,
                
                HOME_SB_RK,
                HOME_SB_PTS_MP,
                HOME_SB_W_MP, 
                HOME_SB_D_MP,
                HOME_SB_L_MP,
                HOME_SB_GD_MP,
                
                AWAY_SB_RK,
                AWAY_SB_PTS_MP,
                AWAY_SB_W_MP,
                AWAY_SB_D_MP,
                AWAY_SB_L_MP,
                AWAY_SB_GD_MP
                
                )
            Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', # mesma qtde de variaveis

            row['GAME_Date'],
            row['GAME_Time'],
            row['WEEK_DAY'],
                       
            row['TEAM_AWAY'],
            row['TEAM_HOME'],
            row['RESULT_GAME'],
                       
            row['SEASON'],
            row['SEASON_BEFORE'],
                       
            row['HOME_SB_RK'],
            row['HOME_SB_PTS_MP'],
            row['HOME_SB_W_MP'],
            row['HOME_SB_D_MP'],
            row['HOME_SB_L_MP'],
            row['HOME_SB_GD_MP'],
                       
            row['AWAY_SB_RK'],
            row['AWAY_SB_PTS_MP'],
            row['AWAY_SB_W_MP'],
            row['AWAY_SB_D_MP'],
            row['AWAY_SB_L_MP'],
            row['AWAY_SB_GD_MP']
        )

    conn.commit()
    cursor.close()

In [122]:
# Function to truncate a table
def SQLTruncate(TableName):
    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    cursor.execute(f'''
    
                   TRUNCATE TABLE {TableName}
                     
                   ''')
    conn.commit()
    cursor.close()

In [123]:
SQLTruncate('TableMatchDetailsFull')

In [124]:
SQLInsertTable(df_MatchDetailsV04)